### MAIN CELL

In [7]:
# main cell
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Read and show crypto terminology table
df = pd.read_csv('crypto-terminology.csv', index_col=[0,1])
#df.dropna()

# Read and show bitcoin table
df = pd.read_csv('bitcoin.csv')
#df.head()

# Convert the 'date' column to a datetime object
df['date'] = pd.to_datetime(df['date'])

# Extract date-related features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.weekday

# Verify if the missing value has been handled
# print("---------")
# print(df['market_cap'].isnull().sum())

# Create lag features for the 'price' column
for lag in range(1, 4):  # Create lag of 1 day, 2 days, and 3 days
    df[f'price_lag_{lag}'] = df['price'].shift(lag)
    
# Calculate rolling window statistics for 'price' and 'total_volume'
window_sizes = [3, 7]  # Example window sizes of 3 and 7 days
for window in window_sizes:
    df[f'price_rolling_mean_{window}'] = df['price'].rolling(window=window).mean()
    df[f'price_rolling_std_{window}'] = df['price'].rolling(window=window).std()
    df[f'volume_rolling_mean_{window}'] = df['total_volume'].rolling(window=window).mean()
    df[f'volume_rolling_std_{window}'] = df['total_volume'].rolling(window=window).std()
#df

# Drop unnecessary columns
df.drop(columns=['coin_name'], inplace=True)  # coin_name has only one value(it's unnecessary)
# Perform a correlation analysis
correlation_matrix = df.corr()
# Display the first few rows of the modified DataFrame and the correlation matrix
#df.head(), missing_values, correlation_matrix['price'].sort_values(ascending=False)
#this is helping to see which values might have strong influence on the price of Bitcoin.

# The dependent variable
y = df['price']
# The independent variables
X = df.drop(['price'], axis=1)


# Check for missing values
missing_values = df.isnull().sum()


# Check for missing values in 'market_cap' and impute with the mean
df['market_cap'] = df['market_cap'].fillna(df['market_cap'].mean())
print(df.isnull().sum())

date                     0
price                    0
total_volume             0
market_cap               0
year                     0
month                    0
day                      0
weekday                  0
price_lag_1              1
price_lag_2              2
price_lag_3              3
price_rolling_mean_3     2
price_rolling_std_3      2
volume_rolling_mean_3    2
volume_rolling_std_3     2
price_rolling_mean_7     6
price_rolling_std_7      6
volume_rolling_mean_7    6
volume_rolling_std_7     6
dtype: int64


### DEVELOPMENT CELL

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Read and show crypto terminology table
df = pd.read_csv('crypto-terminology.csv', index_col=[0,1])
#df.dropna()

# Read and show bitcoin table
df = pd.read_csv('bitcoin.csv')
#df.head()

bull_run_periods = [
    ('2011-11-01', '2013-04-01'),
    ('2015-08-01', '2017-12-01'),
    ('2020-09-01', '2021-11-01'),
    # this one is in the future
    ('2024-12-01', '2026-02-01'),
]

# Convert the 'date' column to a datetime object
df['date'] = pd.to_datetime(df['date'])

df['bull_run'] = 0
print(df['bull_run'])

# Loop over the bull run periods and update the 'bull_run' column accordingly
for start, end in bull_run_periods:
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)
    df.loc[(df['date'] >= start_date) & (df['date'] <= end_date), 'bull_run'] = 1


# Extract date-related features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.weekday

# Verify if the missing value has been handled
# print("---------")
# print(df['market_cap'].isnull().sum())

# Create lag features for the 'price' column
for lag in range(1, 4):  # Create lag of 1 day, 2 days, and 3 days
    df[f'price_lag_{lag}'] = df['price'].shift(lag)
    
# Calculate rolling window statistics for 'price' and 'total_volume'
window_sizes = [3, 7]  # Example window sizes of 3 and 7 days
for window in window_sizes:
    df[f'price_rolling_mean_{window}'] = df['price'].rolling(window=window).mean()
    df[f'price_rolling_std_{window}'] = df['price'].rolling(window=window).std()
    df[f'volume_rolling_mean_{window}'] = df['total_volume'].rolling(window=window).mean()
    df[f'volume_rolling_std_{window}'] = df['total_volume'].rolling(window=window).std()
#df

# Drop unnecessary columns
df.drop(columns=['coin_name'], inplace=True)  # coin_name has only one value(it's unnecessary)
# Perform a correlation analysis
correlation_matrix = df.corr()
# Display the first few rows of the modified DataFrame and the correlation matrix
#df.head(), missing_values, correlation_matrix['price'].sort_values(ascending=False)
#this is helping to see which values might have strong influence on the price of Bitcoin.

# The dependent variable
y = df['price']
# The independent variables
X = df.drop(['price'], axis=1)


# Check for missing values
missing_values = df.isnull().sum()


# Check for missing values in 'market_cap' and impute with the mean
df['market_cap'] = df['market_cap'].fillna(df['market_cap'].mean())

df['price_lag_1'] = df['price_lag_1'].fillna(df['price_lag_1'].mean())
df['price_lag_2'] = df['price_lag_2'].fillna(df['price_lag_2'].mean())
df['price_lag_3'] = df['price_lag_3'].fillna(df['price_lag_3'].mean())

df['price_rolling_mean_3'] = df['price_rolling_mean_3'].fillna(df['price_rolling_mean_3'].mean())
df['price_rolling_mean_7'] = df['price_rolling_mean_7'].fillna(df['price_rolling_mean_7'].mean())

df['price_rolling_std_3'] = df['price_rolling_std_3'].fillna(df['price_rolling_std_3'].mean())
df['price_rolling_std_7'] = df['price_rolling_std_7'].fillna(df['price_rolling_std_7'].mean())

df['volume_rolling_mean_3'] = df['volume_rolling_mean_3'].fillna(df['volume_rolling_mean_3'].mean())
df['volume_rolling_mean_7'] = df['volume_rolling_mean_7'].fillna(df['volume_rolling_mean_7'].mean())

df['volume_rolling_mean_3'] = df['volume_rolling_mean_3'].fillna(df['volume_rolling_std_3'].mean())
df['volume_rolling_mean_7'] = df['volume_rolling_mean_7'].fillna(df['volume_rolling_std_7'].mean())

df['volume_rolling_std_3'] = df['volume_rolling_std_3'].fillna(df['volume_rolling_std_3'].mean())
df['volume_rolling_std_7'] = df['volume_rolling_std_7'].fillna(df['volume_rolling_std_7'].mean())

#print(df.isnull().sum())

# Calculate IQR
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

# Define bounds for what is considered an outlier
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# If you want to log-transform the 'price' to reduce the impact of outliers:
df_log_transformed = df.copy()
df_log_transformed['price'] = np.log1p(df['price'])


# Saving the DataFrame as a CSV file
df.to_csv('bitcoin_modified.csv', index=False)

# The path to the saved CSV file
csv_file_path = '/mnt/data/bitcoin_modified.csv'
#csv_file_path



date                     0
price                    0
total_volume             0
market_cap               0
year                     0
month                    0
day                      0
weekday                  0
price_lag_1              0
price_lag_2              0
price_lag_3              0
price_rolling_mean_3     0
price_rolling_std_3      0
volume_rolling_mean_3    0
volume_rolling_std_3     0
price_rolling_mean_7     0
price_rolling_std_7      0
volume_rolling_mean_7    0
volume_rolling_std_7     0
dtype: int64


### TESTING CELL

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

# Read and show crypto terminology table
df = pd.read_csv('crypto-terminology.csv', index_col=[0,1])
#df.dropna()

# Read and show bitcoin table
df = pd.read_csv('bitcoin.csv')
#df.head()

bull_run_periods = [
    ('2011-11-01', '2013-04-01'),
    ('2015-08-01', '2017-12-01'),
    ('2020-09-01', '2021-11-01'),
    # this one is in the future
    ('2024-12-01', '2026-02-01'),
]

# Convert the 'date' column to a datetime object
df['date'] = pd.to_datetime(df['date'])

df['bull_run'] = 0
print(df['bull_run'])

# Loop over the bull run periods and update the 'bull_run' column accordingly
for start, end in bull_run_periods:
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)
    df.loc[(df['date'] >= start_date) & (df['date'] <= end_date), 'bull_run'] = 1


# Extract date-related features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.weekday

# Verify if the missing value has been handled
# print("---------")
# print(df['market_cap'].isnull().sum())

# Create lag features for the 'price' column
for lag in range(1, 4):  # Create lag of 1 day, 2 days, and 3 days
    df[f'price_lag_{lag}'] = df['price'].shift(lag)
    
# Calculate rolling window statistics for 'price' and 'total_volume'
window_sizes = [3, 7]  # Example window sizes of 3 and 7 days
for window in window_sizes:
    df[f'price_rolling_mean_{window}'] = df['price'].rolling(window=window).mean()
    df[f'price_rolling_std_{window}'] = df['price'].rolling(window=window).std()
    df[f'volume_rolling_mean_{window}'] = df['total_volume'].rolling(window=window).mean()
    df[f'volume_rolling_std_{window}'] = df['total_volume'].rolling(window=window).std()
#df

# Drop unnecessary columns
df.drop(columns=['coin_name'], inplace=True)  # coin_name has only one value(it's unnecessary)
# Perform a correlation analysis
correlation_matrix = df.corr()
# Display the first few rows of the modified DataFrame and the correlation matrix
#df.head(), missing_values, correlation_matrix['price'].sort_values(ascending=False)
#this is helping to see which values might have strong influence on the price of Bitcoin.

# The dependent variable
y = df['price']
# The independent variables
X = df.drop(['price'], axis=1)


# Check for missing values
missing_values = df.isnull().sum()


# Check for missing values in 'market_cap' and impute with the mean
df['market_cap'] = df['market_cap'].fillna(df['market_cap'].mean())

df['price_lag_1'] = df['price_lag_1'].fillna(df['price_lag_1'].mean())
df['price_lag_2'] = df['price_lag_2'].fillna(df['price_lag_2'].mean())
df['price_lag_3'] = df['price_lag_3'].fillna(df['price_lag_3'].mean())

df['price_rolling_mean_3'] = df['price_rolling_mean_3'].fillna(df['price_rolling_mean_3'].mean())
df['price_rolling_mean_7'] = df['price_rolling_mean_7'].fillna(df['price_rolling_mean_7'].mean())

df['price_rolling_std_3'] = df['price_rolling_std_3'].fillna(df['price_rolling_std_3'].mean())
df['price_rolling_std_7'] = df['price_rolling_std_7'].fillna(df['price_rolling_std_7'].mean())

df['volume_rolling_mean_3'] = df['volume_rolling_mean_3'].fillna(df['volume_rolling_mean_3'].mean())
df['volume_rolling_mean_7'] = df['volume_rolling_mean_7'].fillna(df['volume_rolling_mean_7'].mean())

df['volume_rolling_mean_3'] = df['volume_rolling_mean_3'].fillna(df['volume_rolling_std_3'].mean())
df['volume_rolling_mean_7'] = df['volume_rolling_mean_7'].fillna(df['volume_rolling_std_7'].mean())

df['volume_rolling_std_3'] = df['volume_rolling_std_3'].fillna(df['volume_rolling_std_3'].mean())
df['volume_rolling_std_7'] = df['volume_rolling_std_7'].fillna(df['volume_rolling_std_7'].mean())

#print(df.isnull().sum())

# Calculate IQR
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
# Define bounds for what is considered an outlier
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
# log-transform the 'price' to reduce the impact of outliers:
df_log_transformed = df.copy()
df_log_transformed['price'] = np.log1p(df['price'])

# Assuming 'X' is set of features and 'y' is target variable
# For time-series,  it need to split the data in a way that respects the time order
# Decide on the size of the test set
test_size = 0.2  # 20%
# Compute the index at which to split
split_index = int((1 - test_size) * len(df))
# Split the features and target variable into training/testing sets
feature_train, feature_test = X[:split_index], X[split_index:]
price_train, price_test = y[:split_index], y[split_index:]

# Saving the DataFrame as a CSV file
df.to_csv('bitcoin_modified.csv', index=False)

# The path to the saved CSV file
csv_file_path = '/mnt/data/bitcoin_modified.csv'
#csv_file_path



0       0
1       0
2       0
3       0
4       0
       ..
3306    0
3307    0
3308    0
3309    0
3310    0
Name: bull_run, Length: 3311, dtype: int64
